In [19]:
import pandas as pd
import yfinance as yf

from datetime import date
from dateutil.relativedelta import relativedelta
import json

In [20]:
with open('../data/tickers.json') as f:
    tickers = json.load(f)

Get ticker symbols to the list

In [21]:
tickers = [ticker for ticker in tickers.keys()]

Set date range

In [22]:
end_date = date.today()
start_date = end_date - relativedelta(months=6)

In [23]:
tickers = ['AAPL', 'MSFT']

df_list_close = []
df_list_volume = []

for ticker in tickers:

    data = yf.download(ticker, start='2024-01-01', end='2025-01-01', progress=False)
    # Dodavanje kolone za ticker
    data['ticker'] = ticker
    
    # Izračunavanje postotne promene za close_price i volume
    data['close_price_change'] = data['Close'].pct_change()   # Procenat promene cene
    data['volume_change'] = data['Volume'].pct_change()   # Procenat promene volumena

    # Dodavanje samo potrebnih kolona u listu, koristeći indeks umesto 'date' kolone
    df_list_close.append(data[['close_price_change']].rename(columns={'close_price_change': f'{ticker}_close_change'}))
    df_list_volume.append(data[['volume_change']].rename(columns={'volume_change': f'{ticker}_volume_change'}))

# Kombinovanje svih DataFrame-ova za close_price_change i volume_change
final_close = pd.concat(df_list_close, axis=1)
final_volume = pd.concat(df_list_volume, axis=1)

# Spajanje close_price_change i volume_change DataFrame-a
final_df = pd.concat([final_close, final_volume], axis=1)

# Resetovanje indeksa da bi 'date' postao obična kolona
final_df.reset_index(inplace=True)

# Postavljanje tačnog redosleda kolona
final_df = final_df[['Date', 'AAPL_close_change', 'AAPL_volume_change', 'MSFT_close_change', 'MSFT_volume_change']]

# Prikazivanje rezultata
final_df


Price,Date,AAPL_close_change,AAPL_volume_change,MSFT_close_change,MSFT_volume_change
Ticker,,,,,
0,2024-01-02,NaN,NaN,NaN,NaN
1,2024-01-03,-0.007487,-0.291848,-0.000728,-0.086113
2,2024-01-04,-0.012700,0.232290,-0.007178,-0.094526
3,2024-01-05,-0.004013,-0.134479,-0.000516,0.004091
4,2024-01-08,0.024175,-0.050700,0.018872,0.102301
...,...,...,...,...,...
247,2024-12-24,0.011478,-0.431342,0.009374,-0.625924
248,2024-12-26,0.003176,0.172260,-0.002777,0.143723
249,2024-12-27,-0.013242,0.555059,-0.017302,1.211040


In [ ]:
tickers = ['AAPL', 'MSFT']
all_data = []

for ticker in tickers:
    data = yf.download(ticker, start='2024-01-01', end='2025-01-01', progress=False)
    data = data[['Close', 'Volume']].copy()

    data['close_price'] = data['Close']
    data['volume'] = data['Volume']
    data['close_price_change'] = data['Close'].pct_change() * 100
    data['volume_change'] = data['Volume'].pct_change() * 100

    data['ticker'] = ticker
    data.reset_index(inplace=True)

    all_data.append(data[['Date', 'ticker', 'close_price', 'volume', 'close_price_change', 'volume_change']])

final_df = pd.concat(all_data, ignore_index=True)


if isinstance(final_df.columns, pd.MultiIndex):
    final_df.columns = [col[0] for col in final_df.columns]

final_df = final_df.dropna()

final_df


,Date,ticker,close_price,volume,close_price_change,volume_change
1,2024-01-03,AAPL,183.150391,58414500,-0.748750,-29.184846
2,2024-01-04,AAPL,180.824341,71983600,-1.270022,23.228993
3,2024-01-05,AAPL,180.098709,62303300,-0.401291,-13.447924
4,2024-01-08,AAPL,184.452560,59144500,2.417481,-5.070036
5,2024-01-09,AAPL,184.035065,42841800,-0.226343,-27.564186
...,...,...,...,...,...,...
499,2024-12-24,MSFT,438.450836,7164500,0.937386,-62.592351
500,2024-12-26,MSFT,437.233276,8194200,-0.277696,14.372252
501,2024-12-27,MSFT,429.668457,18117700,-1.730156,121.103952
502,2024-12-30,MSFT,423.979858,13158700,-1.323951,-27.371024
